In [1]:
# # convert the <start> <end> to the '$'
# f2 = open('./input2.txt', 'a')

# with open('./input.txt', 'rw') as f:
#     line = f.readline()
#     while line:
#         # print repr(line)
#         if '<start>' in line:
#             f2.write('$\r\n')
#         elif '<end>' not in line:
#             f2.write(line)
#         line = f.readline()

In [2]:
# f2.close()

以上两块生成新的文件，我把start 和end统一改成了'$'，跑一遍就行了

In [3]:
# torch.cuda.is_available()

In [4]:
with open('input.txt', 'r') as f:
    s_all = f.read()
    
# s1 = 
# seq_len = 25
# i = 0
# data = []
# while i + 25 < len(s):
#     data.append(list(s[i:i+25]))
#     i += 25

In [5]:
from collections import Counter
counter = Counter(s_all)
letter2int = {}
i = 0
for k, v in counter.most_common():
    letter2int[k] = i
    i += 1

In [6]:
n_letters = len(letter2int)

In [7]:
len(letter2int)

93

In [8]:
len(s_all)

501470

In [9]:
import torch
import torch.nn as nn
from torch import autograd
from torch import optim
import torch.nn.functional as F
import numpy as np

In [10]:
class LSTM_RNN(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, n_layers, batch_size):
        super(LSTM_RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.n_layers = n_layers

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = 0.3)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # self.hidden = self.init_hidden()

    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim)).cuda())

    def forward(self, sentence,hidden):
        batch_size = sentence.size()[1]
        seq_lens = sentence.size()[0]
        
        embeds = self.word_embeddings(sentence)
        hidden = repackage(hidden)
        lstm_out, hidden = self.lstm(
            embeds.view(seq_lens, batch_size, -1), hidden)
        tag_space = self.hidden2tag(lstm_out.view(seq_lens * batch_size, -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores, hidden
    

In [11]:
def repackage(h):
    if type(h) == autograd.Variable:
        return autograd.Variable(h.data)
    else:
        return tuple(repackage(v) for v in h)

In [12]:
s_train = s_all[:int(len(s_all) * 0.8)]
s_valid = s_all[int(len(s_all) * 0.8):]

In [13]:
int(len(s_all) * 0.8) / 64

6268.375

In [14]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return idxs

def batch_generator(batch_size, chunk_size, s):
    i = 0
    data = []
    while i + chunk_size * batch_size < len(s) - 1:
        lines1 = []
        lines2 = []
        for j in range(batch_size):
            lines1.append(s[i + j * chunk_size: i + (j + 1) * chunk_size])
            lines2.append(s[i + j * chunk_size + 1: i + (j + 1) * chunk_size + 1])
        X = [prepare_sequence(l, letter2int) for l in lines1]
        y = [prepare_sequence(l, letter2int) for l in lines2]
        X = np.array(X).transpose(1,0)
        y = np.array(y).transpose(1,0)
        X = torch.LongTensor(X)
        y = torch.LongTensor(y)
        i += chunk_size * batch_size
        X = autograd.Variable(X.cuda())
        y = autograd.Variable(y.cuda())
        yield X, y

In [15]:
# count = 0
# for a, b in batch_generator(64, 25, s_train):
#     print a.size()
#     count += 1
# print count

In [16]:
len(s_train) / (64 * 25)

250.735

In [26]:
EMBEDDING_DIM = 64
dim = 93
HIDDEN_DIM = 100
batch_size = 64
chunk_size = 25
n_layers = 1
n_epochs = 30

In [27]:
n_train_batch = len(s_train) / (64 * 25)
n_valid_batch = len(s_valid) / (64 * 25)

In [28]:
model = LSTM_RNN(EMBEDDING_DIM, HIDDEN_DIM, n_letters, n_letters, n_layers, batch_size)
model = model.cuda()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [29]:
record = [0, 0, 0]
last_valid_loss = 100000

In [30]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
#inputs = prepare_sequence(training_data[0][0], word_to_ix)
# tag_scores = model(inputs)
# print(tag_scores)

for epoch in range(n_epochs):  # again, normally you would NOT do 300 epochs, it is toy data
    
    losses = 0
    hidden = model.init_hidden(batch_size)
    for X, y in batch_generator(batch_size, chunk_size, s_train):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        # sentence_in = prepare_sequence(sentence, word_to_ix)
        # targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores, hidden = model(X, hidden) # X - 25 * 64,
        tag_scores = tag_scores.view(-1, dim).cuda() 
        y = y.contiguous().view(-1).cuda() # convert label y to contiguous

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        
        loss = loss_function(tag_scores, y)
        loss.backward()
        optimizer.step()
        
        losses += loss.data[0]
    
    losses_valid = 0 
    for X, y in batch_generator(batch_size, chunk_size, s_valid):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()
        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        # sentence_in = prepare_sequence(sentence, word_to_ix)
        # targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores, hidden = model(X, hidden)
        tag_scores = tag_scores.view(-1, dim).cuda()
        y = y.contiguous().view(-1).cuda()

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        
        loss = loss_function(tag_scores, y)
        losses_valid += loss.data[0]
    
    record = record[1:]
    if losses_valid < last_valid_loss:
        record.append(1)
    else:
        record.append(-1)
    if sum(record) == -3:
        print ('early stopped')
        break
    last_valid_loss = losses_valid
        
    print ('training:', losses / n_train_batch)
    print ('valid:', losses_valid / n_valid_batch)

# See what the scores are after training
# inputs = prepare_sequence(training_data[0][0], word_to_ix)
# tag_scores = model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
# print(tag_scores)

training: 2.697504114685813
valid: 2.337134732851481
training: 2.1618541022474975
valid: 2.1641180722438236
training: 1.9993222275410134
valid: 2.091232028766213
training: 1.9095292021860393
valid: 2.040472807887925
training: 1.8505482853646362
valid: 2.0057629499629974
training: 1.8073251107769464
valid: 1.9855350340950972
training: 1.7725826125131932
valid: 1.975498665013118
training: 1.743421329777697
valid: 1.968633805509507
training: 1.7195645958452035
valid: 1.9634771387933683
training: 1.700019123742171
valid: 1.9595601992428635
training: 1.6829565890392049
valid: 1.9555783279872019
training: 1.6678746534368034
valid: 1.9523378055321552
training: 1.654074766235242
valid: 1.9503894074644421
training: 1.6418968632534678
valid: 1.9470262710985415
training: 1.6309851142590734
valid: 1.943496822238736
training: 1.6212407595605383
valid: 1.939254870435558
training: 1.6118535135508671
valid: 1.9347231326332466
training: 1.6031787999618097
valid: 1.931907447582014
training: 1.5952781099

In [31]:
len(s_train)

401176

In [32]:
252 * 64 * 25

403200

In [33]:
252 * 64

16128

In [34]:
X

Variable containing:
   24     1     0  ...     37    18    44
    0     0     8  ...     78     4     7
   25    40     5  ...     58     9    47
       ...          ⋱          ...       
    5    40     3  ...      7     9     4
    4     8     6  ...     50    28     0
    0    21     0  ...      9    13     1
[torch.cuda.LongTensor of size 25x64 (GPU 0)]

In [35]:
y

Variable containing:
  0
  0
  8
 ⋮ 
 18
 44
  0
[torch.cuda.LongTensor of size 1600 (GPU 0)]

In [36]:
tag_scores.size()

torch.Size([1600, 93])

In [37]:
y.size()

torch.Size([1600])

In [38]:
int2letter = {val: key for key, val in letter2int.items()}

In [39]:
int2letter[9]

':'

In [40]:
def prepare_sequence2(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return tensor

In [41]:
def generate(model, prime_str='$', predict_len=100, temperature=0.5):
    hidden = model.init_hidden(1)
    prime_input = prepare_sequence2(prime_str*1, letter2int)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = autograd.Variable(prime_input.cuda())
    inp = inp.view(1,-1)
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        # return top_i
        # Add predicted character to string and use as next input
        temp = int2letter[top_i[0]]
        if temp=="$":
            break
        predicted += temp
        inp = autograd.Variable(prepare_sequence2(temp, letter2int).cuda())
        inp = inp.view(1,-1)

    return predicted

In [49]:
pred = generate(model, prime_str='<', predict_len=1000, temperature=0.75)

In [50]:
print (pred)

<start>
X:14
T:Maightar Mofd | Bc d>e:|2 AGD D2F||
|:gf/e/ | edc BG~G2 GB/A/|dB e/f/a/g/|agc ge|fe edf:|2 fdc AcBA|AG~E2 EGB|1 B2 cA|BG Bd|ge/f/c/ | Bc/B/A/ Bc | BG~G2 Bdf|A>BcB | AGE D2:|2 AGEF GEF|DEFDEFAF DFA|
GFA dcc|Bcd dAG2|fef ef/e/|dB BA|BA B2A d2d|B>A:|2 cd ec|dB AB/c/|dc d2:|
|:fa/f/e/f/ g/e/f/f/f/ ge/g/|1 ange March-30
M:2/4
L:1/8
K:F
def ge|AB ~e3g|ge/d/c/ BA||
<end>
<start>
X:24
T:Cad.
S:Storge dourt>
X:52
T:Ore the McMalan 2 cB|BA BG~G2 G>A:|
|:faf gecA|Bc|ed/c/|dB AB/A/G/E/|EG FD|ED DE|FAFA G>A|
A2A BeedB dBAG |
B2 B2 G>A | B2 BG AB/A/ G>A G>A|ABdg ged|ed BB|AB cA|BA/B/|AB|cd/e/ dB|A>B cB|AFD D3:|
<end>
<start>
X:23
T:Battart>
X:99
T:Bally me norred/B/ AB|cdB ABcA|GBd freengain
S:Sess of March-30
M:2/4
L:1/8
K:D
dB/e/|dB BA|B2 B>c|BG GE:|2 F2 D2:|
<end>
<start>
X:80
T:Paddy Frorickated ||
|:fd de/|dB AB/c/|dB GEF|Agfe fed|1 ed BA|B2 de/d/d/ BA|BAFA|~E3/2d/2 e/d/f/e/d/ c/f/g/e/d/|1 d/e/f/g/|de/f/ | ed ed |
ed/B/ AG |1 GAG F2G2|G2e2 ged|ecA ABd|e>e ef/e/f/|ge/e/ dc|Bd|fdB 